This code has all algorithm which we proposed and we check it using two types of laplacian that is symmetic and unnormalized laplacian. 
And we have set the precision for the edge weigths here so we get desired results for the Petersen graph.
Here we have used edge weigths as (f_i - f_j)^2. 

And we are doing it for just for graphs with edge weigth 1 or 0.


In [3]:
import numpy as np
import scipy as sp
from numpy import linalg as LA
import networkx.linalg.algebraicconnectivity as alg
import collections 
import math
import pandas as pd


Results_ncut = pd.DataFrame(index = ['NUP','NP', 'NS', 'UnUP','UnP', 'UnS'])
Results_conductance = pd.DataFrame(index = ['NUP','NP', 'NS', 'UnUP','UnP', 'UnS'])
Graphs = []

AttributeError: module 'pandas' has no attribute 'compat'

In [ ]:
# This wil get us the similarity matrix W given the graph and the number of vertices n.
# the vertices are number form 0 to n - 1.
def get_matrix2(graph, n):
    
    w = np.zeros((n, n))
   
    for x in graph.keys() :
        for y in graph[x]:
            w[x][y] = 1
            w[y][x] = 1

    return w
    

In [ ]:
def get_unnormalized_laplacian(w):
    
    D = w.sum(axis = 1)
    
    D = np.diag(D)  # The degree matrix D
    
    L = D - w
    
    return L

In [ ]:
def get_normalized_laplacian(w):
    
    D = w.sum(axis = 1)
    D_sqrt = np.sqrt(D)
    
    D_1 = np.reciprocal(D_sqrt)
    D_1 = np.diag(D_1)  
    D = np.diag(D)
    L = np.dot(D_1, np.dot(D - w, D_1))
    
    return L

In [ ]:
# returns the fielder eigen vector given the similartiy matrix W.

def get_fielder(l, n):  #Similairty matix W

    eigvals, eigvecs = sp.linalg.eigh(l)# eigen values and eigen vectors of the unnormalized laplacian D - W 
    #print("Eigenvalues: ",eigvals, "\n")
    eigvecs = np.round( eigvecs, 5)
    eigvals = np.round(eigvals, 4)
    cardinality = 1
    
    fielder_eigval = eigvals[1]
    
    #print(fielder_eigval)
    while(cardinality < n and abs(eigvals[cardinality] - fielder_eigval)<0.000001):
        cardinality = cardinality + 1
        
    fielder = eigvecs.T[1:cardinality, :] #The 2nd smallest eigen vector of the laplacian L = D - W
    
    return fielder



In [ ]:
# Checks weather the graph has to connected components or not i.e is the garph paritioned into two clusters 

def check_partion(w, n): # Similarity matrix W and the number of vetrices n
    
    # We will do bfs and check if the graph is partitioned into two parts or not 
    explored = [0]*n 
    
    queue = collections.deque([0])
    
    queue_size = 1
    
    while(queue_size != 0):
        
        node = queue.popleft()
        queue_size -= 1
        
        if explored[node] == 0:
            
            explored[node] = 1
 
            for i in range(0, n):
                if(w[node][i] != 0):
                    neighbour = i
                    
                    if(explored[neighbour] == 0):
                        queue.append(neighbour)
                        queue_size += 1
    
    # Just need to check if this graph has been partitioned into two parts or not 
    is_partitioned = False
    
    for node in explored:
        if node == 0:
            is_partitioned = True
    
    return is_partitioned  

    


In [ ]:
# If the graph has been paritioned into two parts we need to know what these partitions are this function gives us that
# not only does it outputs the clusters but also a vector whose ith index is 1 if its in the first cluster otherwise 0.

def get_partition(w, n): # the similrity matrix W and the number of vertices n
    
    # We will do bfs from a vertex and find out its conected component the other vertices which are not in the
    # connected component are in the other cluster 
    clusters = [[], []]
    explored = [0]*n
    
    
    queue = collections.deque([0])
    
    queue_size = 1
    
    
    while(queue_size != 0):
        
        node = queue.popleft()
        queue_size -= 1
        
        if explored[node] == 0:
            
            explored[node] = 1
 
            for i in range(0, n):
                if(w[node][i] != 0):
                    neighbour = i
                    
                    if(explored[neighbour] == 0):
                        queue.append(neighbour)
                        queue_size += 1
    
    
    #If it a node is connected to vertex zero then its in cluster 0 otherwise its in cluster 1. 
    for node in range(0, n):
        if explored[node] == 1:
            clusters[0].append(node)
        else:
            clusters[1].append(node)
    
    return clusters, explored



In [ ]:
def get_edge_weights(f, w, n):
    
    new_weight = []
    D = w.sum(axis = 1)
    D_sqrt = np.sqrt(D)
    
    for i in range(0, n):
        for j in range(0, n):
            if(w[i][j] != 0):
                new_weight.append([np.dot((f[:,i] - f[:,j]).T,f[:,i] - f[:,j]), i, j])
    
    new_weight = [ [round(i, 3) for i in elem] for elem in new_weight]
    new_weight = sorted(new_weight, reverse= True)
    

    return new_weight

In [ ]:
# Here we use our current algorithm to partition the graph
# This returns the clusters, the cluster_name vector and the edeges cut
def get_clusters(f, w, n): # the similarity matrix and the number of vertices
    
    new_weight = get_edge_weights(f, w, n)
    
    new_weight.sort(reverse= True)

    new_weight = collections.deque(new_weight)

    new_w = w.copy() # Here we copy the similarity matrix so that when we make changes to new_w nothing happens to the origianl similarty matrix

    edges_cut = []
    while(check_partion(new_w, n) == False):  # keep on removing edges until we have a parition
        edge_remove = new_weight.popleft()

        u = edge_remove[1]
        v = edge_remove[2]

        if(new_w[u][v] != 0):
            edges_cut.append([u,v])

        new_w[u][v] = 0
        new_w[v][u] = 0
        
    
    clusters, cluster_name = get_partition(new_w, n) 
    
    # this part changes the cluster_name from a list to numpy array (This step helps to write easy codes)
    cluster_name = np.asarray(cluster_name)
    cluster_name = np.reshape(cluster_name, (1,n))
    
    
    return clusters, cluster_name, edges_cut 


In [ ]:
def ncut(cluster_name, w, n):
    
    cluster_name = np.asarray(cluster_name)
    cluster_name = np.reshape(cluster_name, (1,n))
    
    mull_1 = cluster_name
    mull_2 = np.ones((1,n)) - mull_1
    
    w_1 = np.dot(mull_1, np.dot(w, np.ones((n,1))))
    w_2 = np.dot(mull_2, np.dot(w, np.ones((n,1))))
    
    
    cut = np.dot(mull_1, np.dot(w,mull_2.T))
    #print(cut, w_1,w_2)
    
    ans = cut*(1/w_1 + 1/w_2)
    ans_1 = cut/min(w_1, w_2)
    
    return ans[0][0], ans_1[0][0] 

    

In [ ]:
def get_cut(clusters, w, n):
    cut = []
    for x in clusters[0]:
        for y in clusters[1]:
            if(w[x][y] != 0):
                cut.append([x,y])
    return cut

In [ ]:
def get_results(l, n, w):
    
    
    fielder = get_fielder(l, n)
    add_col_ncut = []
    add_col_con = []
    
    print("Fielder Eigenvectors:", fielder)
    print("Edge weights:", get_edge_weights(fielder, w, n))
    
    
    clusters, cluster_predict , edges_cut = get_clusters(fielder, w, n) 
    nc, c = ncut(cluster_predict, w, n)
    print("\nFielder Cardinality:", fielder.shape[0])
    print("\nCurrent Updated Algorithm\n")
    print("Ncut value:", nc)
    print("Conductance:", c)
    print("Clusters:", clusters)
    print("Clusters we get using our current algorithm: ", cluster_predict)
    print("The edges removed by the algorithm:",edges_cut)
    print("The edge cut of the clusters:", get_cut(clusters, w, n))
    
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    clusters, cluster_predict , edges_cut = get_clusters(np.reshape(fielder[0,:],(1,n)), w, n) 
    nc, c = ncut(cluster_predict, w, n)
    print("\nCurrent Algorithm\n")
    print("Ncut value:", nc)
    print("Conductance:", c)
    print("Clusters:", clusters)
    print("Clusters we get using our current algorithm: ", cluster_predict)
    print("The edges removed by the algorithm:",edges_cut)
    print("The edge-cut of the clusters:", get_cut(clusters, w, n))
    
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    cluster_predict = [0]*n
    
    clusters = [[], []]
    for i in range(0, n):
        if(fielder[0][i] <= 0):
            cluster_predict[i] = 0
            clusters[0].append(i)
        else:
            cluster_predict[i] = 1
            clusters[1].append(i)
    
    nc, c = ncut(cluster_predict, w, n)
    print("\nSpectral clustering Algorithm\n")
    print("Ncut value:", nc)
    print("Conductance:", c)
    print("Clusters:", clusters)
    print("Clusters we get using our current algorithm: ", cluster_predict)
    print("The edge-cut of the clusters:", get_cut(clusters, w, n))
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    
    return add_col_ncut, add_col_con

In [ ]:
def get_results_1(l, n, w):
    
    
    fielder = get_fielder(l, n)
    add_col_ncut = []
    add_col_con = []
    
    
    clusters, cluster_predict , edges_cut = get_clusters(fielder, w, n) 
    nc, c = ncut(cluster_predict, w, n)
    
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    clusters, cluster_predict , edges_cut = get_clusters(np.reshape(fielder[0,:],(1,n)), w, n) 
    nc, c = ncut(cluster_predict, w, n)
    
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    cluster_predict = [0]*n
    
    clusters = [[], []]
    for i in range(0, n):
        if(fielder[0][i] <= 0):
            cluster_predict[i] = 0
            clusters[0].append(i)
        else:
            cluster_predict[i] = 1
            clusters[1].append(i)
    
    nc, c = ncut(cluster_predict, w, n)
    add_col_ncut.append(nc)
    add_col_con.append(c)
    
    
    return add_col_ncut, add_col_con

In [ ]:
def unnormalized_results(graph, print_results):

    n = graph[1][0]
    graph = graph[0][0]
    w = get_matrix2(graph, n)

    l = get_unnormalized_laplacian(w)
    
    if(print_results == True):
        print("\nUnnormalized results\n")
        return get_results(l, n, w)
    else:
        return get_results_1(l, n, w)

In [ ]:
def normalized_results(graph, print_results):

    n = graph[1][0]
    graph = graph[0][0]
    w = get_matrix2(graph, n)

    l = get_normalized_laplacian(w)
    
    if(print_results == True):
        print("\nNormalized results\n")
        return get_results(l, n, w)
    else:
        return get_results_1(l, n, w)

In [ ]:
def show_results(graph, print_results):
    
    add_col_ncut, add_col_con  = [], []
    
    add_col_ncut, add_col_con = normalized_results(graph[1], print_results)
    un, uc = unnormalized_results(graph[1], print_results)
    add_col_ncut += un
    add_col_con += uc
    
    Results_ncut[graph[0]], Results_conductance[graph[0]] = add_col_ncut, add_col_con
    return 0

In [ ]:
Petersen = ['Petersen', [[{0 : [2, 3],
         1 : [4, 3],
         2 : [0, 4],
         3 : [0, 1],
         4 : [1, 2],
         5 : [6, 0],
         6 : [7, 1],
         7 : [8, 2],
         8 : [9, 3],
         9 : [5, 4]
             }],
        [10]]]

Graphs.append(Petersen)
show_results(Petersen, True)

In [ ]:
Enneahedron = ['Enneahedron', [[{0 : [1, 3, 4],
         1 : [0, 2, 4, 5],
         2 : [1, 3, 6],
         3 : [0, 7, 6, 2],
         4 : [0, 1, 5, 6, 7],
         5 : [1, 4, 6],
         6 : [2, 3, 4, 5, 7],
         7 : [4, 3, 6],
             }],
        [8]]]
Graphs.append(Enneahedron)
show_results(Enneahedron, True)

In [ ]:
Enneahedron_1 = ['Enneahedron_1',[[{0 : [1, 3, 4],
         1 : [0, 2, 4, 6, 5],
         2 : [1, 3, 5],
         3 : [0, 4, 7, 5, 2],
         4 : [0, 1, 3, 6],
         5 : [1, 7, 3, 2],
         6 : [1, 3, 4, 7],
         7 : [6, 5],
             }],
        [8]]]
Graphs.append(Enneahedron_1)
show_results(Enneahedron_1, True)

In [ ]:
Cockroach = ['Cockroach',[[{0 : [1],
         1 : [0, 2],
         2 : [1, 3],
         3 : [2, 4, 8],
         4 : [3, 5, 7],
         5 : [4, 6],
         6 : [5, 7],
         7 : [6, 8, 4],
         8 : [7, 9, 3],
         9 : [8, 10],
         10 : [11]
             }],
        [12]]]
Graphs.append(Cockroach)
show_results(Cockroach, True)

In [ ]:
Cockroach_1 = ['Cockroach_1', [[{0 : [1, 11],
         1 : [0, 2],
         2 : [1, 3],
         3 : [2, 4, 8],
         4 : [3, 5, 7],
         5 : [4, 6],
         6 : [5, 7],
         7 : [6, 8, 4],
         8 : [7, 9, 3],
         9 : [8, 10],
         10 : [11],
         11 : [0]    }],
        [12]]]

Graphs.append(Cockroach_1)
show_results(Cockroach_1, True)

In [ ]:
Path_point  = ['Path_point',[[{0 : [1],
         1 : [0, 2],
         2 : [1, 3],
         3 : [2, 4],
         4 : [3, 5],
         5 : [4, 6],
         6 : [5, 7],
         7 : [6, 8],
         8: [7, 9],
         9: [8, 10],
         10:[9, 11],
         11 :[0,1,2,3,4,5,6,7,8,9,10]
                }],
        [12]]]

Graphs.append(Path_point)
show_results(Path_point, True)

In [ ]:
Two_clusters = ['Two_clusters', [[{0 : [1, 2, 3],
         1 : [0, 2, 3, 4],
         2 : [0, 1, 3],
         3 : [0, 1, 2],
         4 : [5, 6, 7],
         5 : [4, 6, 7],
         6 : [4, 5, 7],
         7 : [4, 5, 6]
        }],
        [8]]]

Graphs.append(Two_clusters)
show_results(Two_clusters, True)

In [ ]:
Discrete_moons = {}

for i in range(0, 19):
    Discrete_moons[i] = [i + 1]
    Discrete_moons[i + 20] = [i + 21]

print(Discrete_moons)

for i in range(0, 10):
    Discrete_moons[i].append(19 - i)
    Discrete_moons[i + 20].append(39 - i)

for i in range(0, 5):
    Discrete_moons[i + 10].append(24 - i)
#print(Discrete_moons)

D_M = ['Discrete_moons',[[Discrete_moons], [40]]]

Graphs.append(D_M)
show_results(D_M, True)

In [ ]:
Double_Tree = {}

for i in range(1,10):
    Double_Tree[i] = [math.floor((i - 1)/2)]
    Double_Tree[i + 10] = [10 + math.floor((i - 1)/2)]

Double_Tree[0] = [10]

D_T = ['Double_Tree',[[Double_Tree], [20]]]

Graphs.append(D_T)
show_results(D_T, True)

In [ ]:
print(D_M)

In [ ]:
Double_Tree1 = {}

for i in range(1,10):
    Double_Tree1[i] = [math.floor((i - 1)/2)]
    Double_Tree1[i + 10] = [10 + math.floor((i - 1)/2)]

Double_Tree1[0] = [10]

for i in range(5,9):
     Double_Tree1[i].append(10+i)
for i in range(0,9):
    Double_Tree1[i].append((i+1)%10)
D_T1 = ['Double_Tree1',[[Double_Tree1], [20]]]

Graphs.append(D_T1)
print(D_T1)
show_results(D_T1, True)



In [ ]:
for graph in Graphs:
    show_results(graph, False)
    
    

In [ ]:
print(Results_conductance.T)


In [ ]:
print(Results_ncut.T)

In [ ]:
print(Results_conductance.loc['UnP', :])

In [ ]:
optimal_cuts = []

for i in range(1, 2**10 - 1):
    cluster_predict = [1]*10
    
    for j in range(0,10):
        if(2**j & i != 0):
            cluster_predict[j] = 0
    
    #print(cluster_predict)
    graph = Petersen[1][0]
    graph = graph[0]
    n = Petersen[1][1]
    n  = n[0]
    #print(ncut(cluster_predict, get_matrix2(graph, n), n))
    if(ncut(cluster_predict, get_matrix2(graph, n), n) == ncut([1,1,1,1,1,0,0,0,0,0], get_matrix2(graph, n), n)):
        
        clusters = [[], []]
        for i in range(0, 10):
            clusters[cluster_predict[i]].append(i)
            
        optimal_cuts.append(clusters)

for x in optimal_cuts:
    print(x)
print(len(optimal_cuts))

In [ ]:
d = Enneahedron_1[1][0][0]
for i in d.keys():
    for j in d[i]:
        if(i < j):
            print(str(i) + ' ' + str(j))

In [ ]:
v = [0.46236,  0.16479, -0.33309, -0.08359,  0.50135, -0.49733,  0.1138,  -0.35984]

for i in range(0,8):
    print(str(i) + ", " + str(v[i]))

In [ ]:
RL_Ncut = Results_ncut.T

In [ ]:
y_SLP = RL_Ncut['SLP'].values
y_UnP = RL_Ncut['UnP'].values
y_RwP = RL_Ncut['RwP'].values
y_SLS = RL_Ncut['SLS'].values
y_UnS = RL_Ncut['UnS'].values
y_RwS = RL_Ncut['RwS'].values